## Study of DE-CIX Frankfurt outage on the 9-10th of April 2018

In [ ]:
using Dates
using Glob
using LargeScaleAnalysis
using PyCall
using PyPlot

In [ ]:
# We use pandas since it has a more timeseries functions
# than DataFrames.jl and TimeSeries.jl.
pd = pyimport("pandas")
DateFormatter = pyimport("matplotlib.dates").DateFormatter
HourLocator = pyimport("matplotlib.dates").HourLocator;

In [ ]:
include("../../ParsimoniousMonitoring/notebooks/thesis.jl")
# tikzplotlib is not working very well with dates, so we use a mpl style.
plt.rc("figure", dpi = 120)
plt.style.use("classicthesis");

### Main

In [ ]:
files = glob("*ndjson.model.json", "/blobs/datasets/thesis/decix_2018-04-09_11/");
models = map(x -> parsefile(DataSegmentationModel, x), files);
length(models)

In [ ]:
timestamps = map(x -> x.time, Iterators.flatten(changepoints.(models)));
datetimes = unix2datetime.(timestamps);

In [ ]:
df = pd.DataFrame(Dict(:timestamp => timestamps), index = datetimes)
df.head()

In [ ]:
fig, ax = subplots(figsize = (4.6, 2))
ax.plot(df.resample("360s").count())
ax.axvspan(DateTime(2018,4,9,19,30), DateTime(2018,4,9,19,50), alpha = 0.35, color = colorblindmap[3], lw = 0, label = "Incident DE-CIX \\#1")
ax.axvspan(DateTime(2018,4,9,23,15), DateTime(2018,4,9,23,45), alpha = 0.35, color = colorblindmap[3], lw = 0)
ax.axvspan(DateTime(2018,4,10,2,0), DateTime(2018,4,10,2,20), alpha = 0.35, color = colorblindmap[4], lw = 0, label = "Incident DE-CIX \\#2")
ax.axvspan(DateTime(2018,4,10,3,35), DateTime(2018,4,10,4,10), alpha = 0.35, color = colorblindmap[4], lw = 0)
ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
ax.xaxis.set_major_locator(HourLocator(byhour=[0,6,12,18]))
ax.set(xlabel = "UTC+0 (GMT)", ylabel = "Changements / 360s")
ax.set_xlim(xmin = DateTime(2018,4,9,6))
ax.grid()
ax.legend(loc = "upper right", fancybox = false)
plt.savefig("../figures/atlas-changefreq-de-cix.pdf")